<a href="https://colab.research.google.com/github/Gaurav-822/RagImplementation/blob/main/RAGImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and Importing Necessary Libraries

## Installing

In [ ]:
!pip install openai
!pip install pinecone-client
!pip install datasets
!pip install langchain
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.2 MB/s e

## Importing

In [ ]:
import openai
import pinecone
import numpy as np

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Setting Up API Keys
Add your keys in the colab to run this

In [ ]:
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API')  # OpenAI API KEY
pinecone.init(api_key=userdata.get('PINECONE_API_KEY'), environment=userdata.get('PINECONE_ENV'))

# Setting Up the ChatBot (GPT-3.5 Turbo)

In [ ]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or userdata.get('OPENAI_API') # openai api key

chat = ChatOpenAI(
    openai_api_key = os.environ["OPENAI_API_KEY"],
    model = 'gpt-3.5-turbo'
)

In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content = "You are a helpful assistant."),
    HumanMessage(content = "Hi AI, how are you?"),
    AIMessage(content = "I'm great thank you. How can I help you?"),
    # testing the chatbot
    HumanMessage(content = "I'd like to understand string theory.")
]

In [ ]:
res = chat(messages)
print(res.content)

Sure, I can help you with that! String theory is a theoretical framework in physics that attempts to explain the fundamental nature of particles and forces in the universe. It suggests that the most basic building blocks of matter are not point-like particles, but tiny, vibrating strings.

According to string theory, these strings can vibrate at different frequencies, and each frequency corresponds to a different particle. For example, a string vibrating at a certain frequency may appear as an electron, while a different frequency may correspond to a photon, the particle of light.

One of the key ideas in string theory is that it requires extra dimensions of space beyond the three dimensions of space and one dimension of time that we typically observe. These extra dimensions are believed to be curled up or compactified on a very small scale, making them invisible to our current observations.

String theory also proposes the existence of different versions called "string theories" or "s

To Deal with Hallucinations, either we can add the data not know to the llm in t's memory directly or via RAG
# Implementation of RAG

## Vectorizing and Embedding

### Pinecone Index

In [ ]:
index_name = 'text'
index = pinecone.Index(index_name)

### Embedding Data for Retrival Augmented Generation
Using a list to store the data for simplicity of this project for now as I don't have the subscription of OpenAI's API to use large dataset for free (the rate limit for embedding excedded when I tried an Hugging face dataset)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
import pandas as pd

embed_model = OpenAIEmbeddings(model = "text-embedding-ada-002")
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here',
    # Add more data here
    'Code Llama is a code generation model built on Llama 2, trained on 500B tokens of code. It supports common programming languages being used today',
    'Coffee Shop Sales Data Analysis (2023): Our coffee shop experienced a year of robust growth in 2023, with overall sales increasing by 15% compared to the previous year.  This was driven by strong performance in both hot and cold coffee sales, as well as a surge in popularity of our new line of artisanal pastries. Additionally, we saw a significant increase in online orders, driven by successful targeted marketing campaigns on social media.'

]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(4, 1536)

### Upserting Pinecone index

In [ ]:
embeddings = []
for i in range(len(texts)):
  embeddings.append((f"embedding_id_{i}", res[i], {'text': texts[i]}))
# embeddings[0]
index.upsert(vectors=embeddings)

# Implementing Retrieval Augmented Generation using Similarity Search

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:62: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Demonstration of the Similiarity Search Operartion

In [ ]:
query = "How much sales increased?"

vectorstore.similarity_search(query, k=3)

[Document(page_content='Coffee Shop Sales Data Analysis (2023): Our coffee shop experienced a year of robust growth in 2023, with overall sales increasing by 15% compared to the previous year.  This was driven by strong performance in both hot and cold coffee sales, as well as a surge in popularity of our new line of artisanal pastries. Additionally, we saw a significant increase in online orders, driven by successful targeted marketing campaigns on social media.'),
 Document(page_content='this is the first chunk of text'),
 Document(page_content='then another second chunk of text is here')]

# Feeding our Chatbot

In [ ]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [ ]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Coffee Shop Sales Data Analysis (2023): Our coffee shop experienced a year of robust growth in 2023, with overall sales increasing by 15% compared to the previous year.  This was driven by strong performance in both hot and cold coffee sales, as well as a surge in popularity of our new line of artisanal pastries. Additionally, we saw a significant increase in online orders, driven by successful targeted marketing campaigns on social media.
this is the first chunk of text
then another second chunk of text is here

    Query: How much sales increased?


# Final Output

In [ ]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

The sales increased by 15% compared to the previous year.
